In [ ]:
from tensorflow.keras.models import load_model

# Load HDF5 model
model = load_model('soil_classification_model.h5')

In [ ]:
test_dir = '/kaggle/input/soil-classification/soil_classification-2025/test'

test_df = pd.read_csv('/kaggle/input/soil-classification/soil_classification-2025/test_ids.csv')
test_df['image_path'] = test_df['image_id'].apply(lambda x: f"{test_dir}/{x}")

test_df.head()

1. Specifies the directory where test images are stored.
2. Reads test_ids.csv file that contains a column named 'image_id'
3. Adds a new column called 'image_path' that combines the test directory with the image file name.
4. This is necessary because flow_from_dataframe needs the full path to locate each image.
5. Displays the first few rows of the test dataframe to confirm that the paths were constructed correctly.

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def load_and_preprocess(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize
    return img_array

# Preprocess all test images
X_test = np.array([load_and_preprocess(path) for path in test_df['image_path']])


* Loops through each image path in test_df.

* Applies the load_and_preprocess function.

* Converts the list of preprocessed images into a NumPy array (X_test) with shape:
(num_images, 224, 224, 3).

In [ ]:
predictions = model.predict(X_test)

# If it's a classification model (e.g., softmax output), get class labels
predicted_labels = np.argmax(predictions, axis=1)

# Add predictions to the DataFrame
test_df['prediction'] = predicted_labels

* Feeds the test images into your trained CNN model.

* The output, predictions, is a 2D array of shape (num_samples, num_classes) — each row contains the softmax probabilities for the 4 soil classes.

* np.argmax selects the index of the maximum value (i.e., the class with the highest probability).

* You now have a 1D array of integers representing the predicted class index for each image (e.g., [0, 2, 1, ...]).

* Adds a new column 'prediction' to test_df that contains the predicted class indices.


In [ ]:
# class_indices is a dict like: {'clay': 0, 'red': 1, 'black': 2, 'alluvial': 3}
class_indices = train_data.class_indices

# Reverse it to map index to class
index_to_class = {v: k for k, v in class_indices.items()}

class_label ={
    0:'Alluvial soil',
    1:'Black Soil',
    2:'Clay soil',
    3:'Red soil'
}

Checking the Mapping of integer labels with Actual Soil Type

Creating a Dictionary of Labels to replace them in submission.csv

In [ ]:
# Map encoded predictions to class labels
test_df['soil_type'] = test_df['prediction'].map(class_label)

# Save the correct columns to CSV
test_df[['image_id', 'soil_type']].to_csv('submission.csv', index=False)

creating the final submission.csv with image_id from test dataset and Predicted label

This submission.csv will be submitted for final evaluation